Intel Extension for Transformers provides a comprehensive suite of Langchain-based extension APIs, including advanced retrievers, embedding models, and vector stores. These enhancements are carefully crafted to expand the capabilities of the original Langchain API, ultimately boosting overall performance. This extension is specifically tailored to enhance the functionality and performance of RAG.

# Prepare Environment

Install intel extension for transformers:

In [5]:
!pip install intel-extension-for-transformers

Install Requirements:

In [ ]:
!git clone https://github.com/intel/intel-extension-for-transformers.git
%cd ./intel-extension-for-transformers/intel_extension_for_transformers/neural_chat/
!pip install -r requirements.txt
%cd ../../../

# Run LLM with Langchain-extension API

We've revamped the Chroma API, enabling users to adjust and fine-tune their settings even after the chatbot has been initialized, offering a more adaptable and user-friendly experience.

In [ ]:
!curl -o Intel_AR_WR.pdf https://d1io3yog0oux5.cloudfront.net/_897efe2d574a132883f198f2b119aa39/intel/db/888/8941/file/412439%281%29_12_Intel_AR_WR.pdf

Chatbot code with Langchain APIs:

In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.vectorstores import Chroma

loader = PyPDFLoader("./Intel_AR_WR.pdf")
langchain_documents = loader.load_and_split()
embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-base-en-v1.5")
knowledge_base = Chroma.from_documents(documents=langchain_documents, embedding=embeddings, persist_directory='./out')
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
pipe = HuggingFacePipeline(pipeline=pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128))
retriever = VectorStoreRetriever(vectorstore=knowledge_base, search_type='mmr', search_kwargs={'k':1, 'fetch_k':5})
retrievalQA = RetrievalQA.from_llm(llm=pipe, retriever=retriever)
result = retrievalQA({"query": "What is IDM 2.0?"})
print(result)

Chatbot code with ITREX Langchain extension APIs:

In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.documents import Document
from intel_extension_for_transformers.langchain.embeddings import HuggingFaceBgeEmbeddings
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from intel_extension_for_transformers.langchain.vectorstores import Chroma
from intel_extension_for_transformers.neural_chat.pipeline.plugins.retrieval.parser.parser import DocumentParser

document_parser = DocumentParser()
input_path="./Intel_AR_WR.pdf"
data_collection=document_parser.load(input=input_path)
documents = []
for data, meta in data_collection:
    doc = Document(page_content=data, metadata={"source":meta})
    documents.append(doc)
# load Intel/bge-base-en-v1.5-sts-int8-static from local
embeddings = HuggingFaceBgeEmbeddings(model_name="./bge-base-en-v1.5-sts-int8-static")
knowledge_base = Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory='./output')
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
pipe = HuggingFacePipeline(pipeline=pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128))
retriever = VectorStoreRetriever(vectorstore=knowledge_base, search_type='mmr', search_kwargs={'k':1, 'fetch_k':5})
retrievalQA = RetrievalQA.from_llm(llm=pipe, retriever=retriever)
result = retrievalQA({"query": "What is IDM 2.0?"})
print(result)

Comparing the execution time, using ITREX Langchain extension APIs can get better performance.

| APIs   |   Execution Time   |
|-------|-------|
| Langchain  | 106.094 sec  |
| ITREX Langchain Extension | 81.429 sec  |


We've specifically designed `ChildParentRetriever` to address challenges in long-context retrieval scenarios. Our strategy involves initially splitting the user-uploaded files into larger chunks, termed 'parent chunks'. Then, these parent chunks are further divided into smaller 'child chunks'. Both child and parent chunks are interconnected using a unique identification ID. This approach enhances the likelihood and precision of matching the user query with a relevant, concise context chunk.

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from intel_extension_for_transformers.neural_chat.pipeline.plugins.retrieval.retrieval_agent import document_transfer, document_append_id
from intel_extension_for_transformers.neural_chat.pipeline.plugins.retrieval.parser.parser import DocumentParser
from intel_extension_for_transformers.langchain.retrievers import ChildParentRetriever

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512)
document_parser = DocumentParser()
input_path="./Intel_AR_WR.pdf"
data_collection=document_parser.load(input=input_path)
langchain_documents = document_transfer(data_collection)
child_documents = text_splitter.split_documents(langchain_documents)
langchain_documents = document_append_id(langchain_documents)
embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-base-en-v1.5")
knowledge_base = Chroma.from_documents(documents=langchain_documents, embedding=embeddings, persist_directory='./parent')
child_knowledge_base = Chroma.from_documents(documents=child_documents, embedding=embeddings, persist_directory='./child')
retriever = ChildParentRetriever(vectorstore=knowledge_base, parentstore=child_knowledge_base)
docs=retriever.get_relevant_documents("What is IDM 2.0?")
print(docs)

Comparing with result using default Langchain retriever, ITREX Langchain extension APIs can get better result.


| Retrieval Type   |   Retrieval Result   |
|-------|-------|
| default  | The Smart Capital strategy helps the company leverage various sources of capital to support investments in manufacturing capacity and fund their IDM 2.0 strategy.  |
| ITREX Langchain Extension | Smart Capital for IDM 2.0 includes aggressive building out of manufacturing shell space, which gives flexibility in how and when to bring additional capacity online based on milestone triggers such as product readiness, market conditions, and customer commitments. It also involves government incentives, Strategic Capacity Investments (SCIP), customer commitments, and external foundries.   |